In [10]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import os
import json
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from nltk.tokenize import sent_tokenize, word_tokenize
sys.path.append("/Users/rohanramesh/Documents/GitHub/Insight_writers/lib/")
from text_processing import ProcessArticle as pa
import suggestions as s
import difflib
from scipy.stats import wilcoxon, sem

In [17]:
# data load
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)
    
# load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)
    
# load writer features
writer_features = pd.read_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_features_USE.pickle')

# load podcasts
top_podcasts = pd.read_pickle('/Users/rohanramesh/Documents/Insight/data_bball_writers/top50podcasts.pickle')


In [14]:
for col in writer_features.columns:
    print(col == 'avg_n_words')
    break

True


In [20]:
writer_features.head(120)

,avg_n_words,avg_neg_sent,avg_neu_sent,avg_pos_sent,avg_neg_sent_var,avg_neu_sent_var,avg_pos_sent_var,avg_n_sentences,avg_n_wordspersentence,avg_n_wordspersent_variability,avg_wordlength,avg_wordlength_var,avg_wordlength_skew,author_list,author_fn
0,2681.350000,0.054043,0.819340,0.126635,0.086493,0.139960,0.115167,146.125000,21.994431,10.687502,4.408409,2.272274,0.989386,steve-kyler,Steve Kyler
1,2637.789474,0.050321,0.858624,0.090417,0.098964,0.153447,0.126745,160.164474,21.808768,13.929073,4.486540,2.457049,1.168685,chris-ballard,Chris Ballard
2,2577.953488,0.049494,0.867080,0.079961,0.096608,0.153289,0.120339,154.237209,21.463349,13.157404,4.520716,2.442506,1.120085,lee-jenkins,Lee Jenkins
3,2391.243243,0.056028,0.824903,0.119087,0.089104,0.139306,0.110893,131.297297,21.916309,10.882959,4.441745,2.311511,0.963884,dennis-chambers,Dennis Chambers
4,2288.380531,0.031264,0.856933,0.111534,0.072845,0.145780,0.133483,125.707965,23.067186,12.701427,4.345722,2.424053,1.286446,jared-zwerling,Jared Zwerling
5,2154.960000,0.061887,0.825074,0.113040,0.092627,0.140608,0.118363,96.580000,27.289474,16.083856,4.548938,2.551031,1.161543,justin-willard,Justin Willard
6,2057.886792,0.053332,0.832232,0.114257,0.108221,0.174830,0.147800,130.075472,21.940964,16.572673,4.567793,2.602091,1.220286,billsimmons,Bill Simmons
7,2043.020408,0.056250,0.854546,0.089205,0.098479,0.148075,0.121684,124.515306,21.447863,12.323328,4.682058,2.463624,1.145495,zachlowe_nba,Zach Lowe
8,1985.897959,0.038959,0.833628,0.127418,0.077193,0.146536,0.133164,107.193878,22.635080,11.920550,4.372536,2.367299,1.035535,alex-kennedy,Alex Kennedy
9,1886.772727,0.043176,0.861178,0.095369,0.090482,0.149968,0.126053,96.772727,25.217095,16.015780,4.470177,2.388765,1.088696,david-aldridge,David Aldridge


In [8]:
# grab config file
# keys[0] apple, keys[1] listen notes
path_teams = '/Users/rohanramesh/Documents/Insight/data_bball_writers/config.txt'
with open(path_teams, 'r') as f:
    tmp = f.readlines()
keys = [i.rstrip() for i in tmp]

In [ ]:
dbname = 'writer_feature_db'
# dbname = 'writer_db'
username = 'rohanramesh' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
con = None
con = psycopg2.connect(database = dbname, user = username)

In [ ]:
# query to load entire db into pandas
sql_query = """
SELECT * FROM writer_feature;
"""
writer_feature_df = pd.read_sql_query(sql_query,con)

In [ ]:
sql_query = """
SELECT AVG("n_words") AS "avg_n_words",
         AVG("neg_sent") AS "avg_neg_sent",
         AVG("neu_sent") AS "avg_neu_sent",
         AVG("pos_sent") AS "avg_pos_sent",
         AVG("neg_sent_var") AS "avg_neg_sent_var",
         AVG("neu_sent_var") AS "avg_neu_sent_var",
         AVG("pos_sent_var") AS "avg_pos_sent_var",
        AVG("n_sentences") AS "avg_n_sentences",
        AVG("n_wordspersentence") AS "avg_n_wordspersentence",
        AVG("n_wordspersent_variability") AS "avg_n_wordspersent_variability",
        AVG("wordlength") AS "avg_wordlength",
        AVG("wordlength_var") AS "avg_wordlength_var",
        AVG("wordlength_skew") AS "avg_wordlength_skew",
        "author_list"
FROM writer_feature
WHERE "n_words" > 100
GROUP BY "author_list"
HAVING COUNT(*) > 25 
ORDER BY AVG("n_words") DESC
"""
writer_feature_subsection = pd.read_sql_query(sql_query,con)
writer_feature_subsection.head(10)
writer_feature_subsection.replace([np.inf, -np.inf], np.nan)
writer_feature_subsection = writer_feature_subsection.dropna(axis=0, how='any')

In [ ]:
author_webnames = writer_feature_subsection['author_list']
author_list = []
for i in author_webnames:
    idx = writer_df['website_name'] == i
#     print(i)
#     print(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])
#     break
    author_list.append(writer_df['Idea Text'][np.where(idx)[0]].tolist()[0])
    
# author_list[41] = 'Jason Concepcion'

In [ ]:
# type something in search key
# new_id = url_format.format('zach', 'lowe')
# new_id = 'https://www.listennotes.com/search/?q=zach%20lowe'
page = requests.get(url_format)
curr_soup = BeautifulSoup(page.text)

In [ ]:
# top_podcasts = {}
# for curr_author in author_list:
#     top_podcasts[curr_author] = []
#     r = word_tokenize(curr_author)
#     page_idx = 0
#     while page_idx < 50:
# #         url = ("https://listennotes.p.mashape.com/api"
# #             "/v1/search?offset={}&q={}+{}&sort_by_date=0&type=episode".format(page_idx,r[0], r[1]))
# #         response = requests.get(url,
# #           headers={
# #             "X-Mashape-Key": "PUT IN KEY",
# #             "Accept": "application/json"
# #           }
# #         )
#         a = response.content
#         b = json.loads(a)
#         top_podcasts[curr_author].append(b)
#         page_idx += 10
#         time.sleep(random.uniform(0.5,2))
#         print(url)


In [ ]:
# These code snippets use an open-source library. http://unirest.io/python
response = requests.get("https://listennotes.p.mashape.com/api/v1/search?offset=0&q=zach+lowe&sort_by_date=0&type=episode",
  headers={
    "X-Mashape-Key": "KEYINEMAIL",
    "Accept": "application/json"
  }
)

In [ ]:
# print(ast.literal_eval(a))
a = response.content
b = json.loads(a)

In [ ]:
# for each podcast lets identify fields to search
author_mentions = np.zeros([len(author_list), len(author_list)])
for i in range(0,len(author_list)):
    curr_author = author_list[i]
    # curr_page = 0
    for curr_page in range(0,4):
    # curr_pod = 0
        for curr_pod in range(0,np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])):
            curr_info = top_podcasts[curr_author][curr_page]['results'][curr_pod]
            podcast_keys = curr_info.keys()
            matches = []
            for curr_key in podcast_keys:
                for j in range(0,len(author_list)):
                    curr_authorcomp = author_list[j]
                    if curr_authorcomp == 'Jason Concepcion/netw3rk':
                        curr_authorcomp = 'Jason Concepcion'
                    search_terms = curr_authorcomp
                    #     matches = []
                    if isinstance(curr_info[curr_key], str):
                #         print(curr_info[curr_key])
                        last_name = word_tokenize(search_terms)[1]
                #         print(last_name)
                    #     for word in search_terms.split(" "):
                    #         print(word)
                        if last_name in curr_info[curr_key]:
                            author_mentions[i,j] = 1
                        

In [ ]:
# curr_info
# top_podcasts[curr_author][curr_page]
# np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])
def match_author_names(writer_df, author):
    wa = writer_df['website_name'].tolist()
    al = writer_df['Idea Text'].tolist()
    if author in wa:
        return al[wa.index(author)]
    elif author in al:
        return wa[al.index(author)]
    else:
        print('improper author name')
            

In [ ]:
plt.imshow(author_mentions)

In [ ]:
r = 'Zach Lowe'
idx = np.where(author_mentions[author_list.index(r)])
b = [author_list[i] for i in idx[0] if author_list[i] != r]
b

In [ ]:
author_sugg = s.give_author_suggestion_from_author(writer_features, 'zachlowe_nba')
print(author_sugg['authors'].tolist())

In [ ]:
# iterate each author and see how many times match in podcasts in either direction
# for i in range(0,len(author_list)):
website_author_names = writer_df['website_name'].tolist()
matched_fraction = []
nonmatch_fraction = []
for ii in range(0,len(author_list)):
# ii = 7
    if np.sum(author_mentions[ii,:]) <= 1:
        print(author_list[ii])
        continue
    curr_author = author_list[ii]
    website_name = match_author_names(writer_df, curr_author)
    author_sugg = s.give_author_suggestion_from_author(writer_features, website_name)
    author_sugg_list = author_sugg['authors'].tolist()
    matched_authors = author_sugg_list[0:6]
#     print(matched_authors)
    nonmatched_authors = [i for i in website_author_names if i not in matched_authors]
    # to remove same author
    matched_authors = matched_authors[1:]
    # get n for matched authors
    n_match = []
    for i in matched_authors:
        idx1 = author_list.index(curr_author)
        idx2 = author_list.index(match_author_names(writer_df, i))
        val = np.max([author_mentions[idx1,idx2], author_mentions[idx2,idx1]])
        n_match.append(val)
    matched_fraction.append(np.sum(n_match)/len(n_match))
    n_nonmatch = []
    for i in nonmatched_authors:
        if match_author_names(writer_df, i) in author_list:
            idx1 = author_list.index(curr_author)  
            idx2 = author_list.index(match_author_names(writer_df, i))
            val = np.max([author_mentions[idx1,idx2], author_mentions[idx2,idx1]])
            n_nonmatch.append(val)
    nonmatch_fraction.append(np.sum(n_nonmatch)/len(n_nonmatch))


In [ ]:
plt.hist(nonmatch_fraction)
plt.show()
plt.hist(matched_fraction)
plt.show()
print(np.mean(matched_fraction))
print(np.mean(nonmatch_fraction))
# (np.mean(matched_fraction)-np.mean(nonmatch_fraction))/np.mean(matched_fraction)
# print((np.mean(nonmatch_fraction)*100)/(np.mean(matched_fraction)*100-np.mean(nonmatch_fraction)*100))
print(np.mean(matched_fraction)/np.mean(nonmatch_fraction))
a,pval = wilcoxon(matched_fraction, nonmatch_fraction)
pval

In [ ]:
# plt.boxplot([matched_fraction, nonmatch_fraction])
plt.bar(range(0,2),[np.mean(matched_fraction)*100, np.mean(nonmatch_fraction)*100], 0.75, 
        yerr=[sem(matched_fraction)*100, sem(nonmatch_fraction)*100], tick_label= ['Suggested', 'Not suggested'])
plt.ylabel('Percent of writers to do podcasts together')
pval

In [ ]:
curr_page = 2
curr_author = 'Zach Lowe'
curr_pod = 8
# for curr_pod in range(0,np.min([9,len(top_podcasts[curr_author][curr_page]['results'])])):
curr_info = top_podcasts[curr_author][curr_page]['results'][curr_pod]
podcast_keys = curr_info.keys()
matches = []
for curr_key in podcast_keys:
    for j in range(0,len(author_list)):
        curr_authorcomp = author_list[j]
        search_terms = curr_authorcomp
        #     matches = []
        if isinstance(curr_info[curr_key], str):
#             print(curr_info[curr_key])
            last_name = word_tokenize(search_terms)[1]
    #         print(last_name)
        #     for word in search_terms.split(" "):
        #         print(word)
            if last_name in curr_info[curr_key]:
    #                 author_mentions[i,j] = 1
                print(curr_authorcomp)
                print('woo')
                

In [15]:
author_list

NameError: name 'author_list' is not defined

In [16]:
matched_authors

NameError: name 'matched_authors' is not defined